In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def build_model(hp):
    units = hp.Int(name='units',min_value=16,max_value=64,step=16)
    model=keras.Sequential([
        layers.Dense(units,activation='relu'),
        layers.Dense(10,activation='softmax')
    ])
    optimizer=hp.Choice(name='optimizer',values=['adagrad','adam'])
    model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

In [3]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self,num_classes):
        self.num_classes=num_classes
        
    def build(self,hp):
        units = hp.Int(name='units',min_value=16,max_value=64,step=16)
        model=keras.Sequential([
                layers.Dense(units,activation='relu'),
                layers.Dense(10,activation='softmax')
                ])
        optimizer=hp.Choice(name='optimizer',values=['adagrad','adam'])
        model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
        return model
        
    
hypermodel=SimpleMLP(num_classes=10)

In [4]:
tuner=kt.BayesianOptimization(build_model,
                             objective='val_accuracy',
                             max_trials=5
                             )

In [6]:
#Load cifar10 dataset from keras

(xtrain,ytrain),(xtest,ytest)=keras.datasets.cifar10.load_data()
xtrain=xtrain.reshape((-1,32*32*3)).astype('float32')/255
xtest=xtest.reshape((-1,32*32*3)).astype('float32')/255
xtrain_full=xtrain[:]
ytrain_full=ytrain[:]
num_val_samples=10000
xtrain,xval=xtrain[:-num_val_samples],xtrain[-num_val_samples:]
ytrain,yval=ytrain[:-num_val_samples],ytrain[-num_val_samples:]

callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)]

tuner.search(xtrain,ytrain,batch_size=128,epochs=20,
            validation_data=(xval,yval),
            callbacks=callbacks,verbose=2)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.35370001196861267

Best val_accuracy So Far: 0.40639999508857727
Total elapsed time: 00h 02m 09s
